In [390]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from selenium import webdriver
from selenium.webdriver import ActionChains
import time
import dateparser
import datetime
from pathlib import Path
import json

In [391]:
# Import GeckoDriverManager module.
from webdriver_manager.firefox import GeckoDriverManager

# Install the GeckoDriverManager to run FireFox web browser (for the first time!)
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

# Once driver is installed, use this line:
driver = webdriver.Firefox(executable_path='./geckodriver.exe')


In [392]:
# Bandcamp Web Scraper Data format information

'''

scrape_data = pd.DataFrame(columns=['Release Title', 
                                    'Artist Name', 
                                    'Artist Location',
                                    'Release Date',
                                    'Release URL',
                                    'Release Genre',
                                    'Release Sub-Genre',
                                    'Search Format',
                                    'Search Week',
                                    'Search Category',
                                    'All Lyrics',
                                    'Number of Tracks'
                                    'Track Info',
                                    'Number of Fans',
                                    'Tags',
                                    'Scrape Date'])
                                    
### dictionary entry for track listing

track_info_entry = {
    'Track Title' : '',
    'Track Lyrics' : '',
    'Track Number' : '#',
    'Track Duration': '##:##'
}
                                    
'''

# valid weeks: [-1,0,678,677,676,675,674,673]
# today, this week, last week, 2, 3, 4, 5, 6 weeks ago
WEEK_DICT = {
    -1:'today',
    0:'this week',
    678:'last week',
    677:'2 weeks ago',
    676:'3 weeks ago',
    675:'4 weeks ago',
    674:'5 weeks ago',
    673:'6 weeks ago'
}

# scrape dataframe list to collect release entries
scrape_data = []

# all URLS collected so far (don't collect duplicate albums for multiple searches)
# load 'all_URLS_collected.txt' file if it exists
all_URLS_collected = []
URLS_file = Path("all_URLS_collected.txt")
if URLS_file.is_file():
    with open("all_URLS_collected.txt", "r") as fp:
        all_URLS_collected = json.load(fp)
    

In [393]:
# scrape bandcamp release page function
def scrape_release_page(URL,entry,output=False):
    
    entry = entry.copy()
    
    entry['Release URL'] = URL
    entry['Scrape Date'] = datetime.datetime.now().date()
    
    # don't add entry if it already exists in scrape dataframe
    if URL in all_URLS_collected:
        return
    else:
        all_URLS_collected.append(URL)
    
    if output==True:
        print('retrieving info for ' + URL + ' ...')

    headers = {'user-agent': 'Mozilla/5.0'}
    response = requests.get(URL,headers=headers)
    
    if response.status_code == 200:

        soup = bsoup(response.text, 'lxml')
        
        # Get title and artist name
        title = soup.find('meta',{'name':'title'})['content'].split(', by ')
        entry['Release Title'] = title[0]
        entry['Artist Name'] = title[1]
        
        # Get artist location
        entry['Artist Location'] = soup.find('span',{'class':'location secondaryText'}).string
        
        # Get release date
        release_date = soup.find('div',{'class':'tralbumData tralbum-credits'}).contents[0].string.strip('"').strip(' ').replace('releases','').replace('released','')
        release_date = dateparser.parse(release_date)
        entry['Release Date'] = release_date
        
        # Get tags
        tags = soup.find_all('a',{'class':'tag'})
        tags_list = []
        for tag in tags:
            tags_list.append(tag.string)
        entry['Tags'] = tags_list
        
        # If no genre in entry, leave blank
        if 'Release Genre' not in entry:
            entry['Release Genre'] = 'N/A'
            
        # If no subgenre in entry, leave blank
        if 'Release Sub-Genre' not in entry:
            entry['Release Sub-Genre'] = 'N/A'            
        
        # Get track information
        entry['Track Info'] = []
        #tracks = soup.find('table',{'id':'track_table'}).findChildren('tr', recursive=False)
        tracks = soup.find('table',{'id':'track_table'}).find_all('tr',{'class':'track_row_view'})
        
        all_lyrics = ''
        for track in tracks:
            # Track number
            track_num = int(track.find('td',{'class':'track-number-col'}).div.string.strip('.'))

            # Track title
            track_title = track.find('span',{'class':'track-title'})
            if track_title:
                track_title = track_title.string
            else:
                track_title = track.find('div',{'class':'title'}).span.string
        
            # Track duration
            track_duration = track.find('span',{'class':'time secondaryText'})
            if track_duration:
                track_duration = track_duration.string.replace('\n','').strip(' ')
            else:
                track_duration = 'N/A'  
                
            # Track lyrics
            track_lyrics = 'N/A'
            
            track_lyric_link = track.find('div',{'class':'info_link'}).a['href']            
            if track_lyric_link and '#lyrics' in track_lyric_link:
                lyric_tag = track.findNext('tr',{'class':'lyricsRow'})
                track_lyrics = lyric_tag.find('td',{'colspan':'4'}).div.string.strip('"').replace('\r\n','\n')
                all_lyrics += '\n' + track_lyrics
            
            # Add track object to tracks list
            track_obj = {'Track Title': track_title, 'Track Lyrics': track_lyrics,
                        'Track Number': track_num, 'Track Duration': track_duration}
            entry['Track Info'].append(track_obj)

        if all_lyrics != '':
            entry['All Lyrics'] = all_lyrics
        else:
            entry['All Lyrics'] = 'N/A'
        
        # Number of Tracks
        entry['Number of Tracks'] = len(entry['Track Info'])
        
        # Popularity Index
        entry['Number of Fans'] = 'N/A'
        driver.get(URL)
        foundAllFans = False
        fan_pages_searched = 0
        while foundAllFans == False and fan_pages_searched < MAX_FAN_PAGES:
            try:
                more_thumbs = driver.find_element_by_xpath('//a[@class="more-thumbs"]')
                fan_pages_searched += 1
                if 'display: none' in more_thumbs.get_attribute("style"):
                    foundAllFans = True
            except:
                foundAllFans = True
        
        if fan_pages_searched == MAX_FAN_PAGES:
            entry['Number of Fans'] = '>' + str(MAX_FANS)
        else:
            try:
                parentElement = driver.find_element_by_xpath('//div[@class="no-writing"]')
                elementList = parentElement.find_elements_by_tag_name("a")
                entry['Number of Fans'] = len(elementList)                   
            except:
                entry['Number of Fans'] = 0
                
        if output == True:
            #pprint(entry)
            #print()
            pass
        
        # add entry to scrape data
        scrape_data.append(entry)
    
    pass

In [394]:
# scrape search page function
# scrape the number of pages in the given search URL
def scrape_search_page(URL,num_pages=NUM_PAGES,output=False):

    # read parameters from link
    parameters = URL.split('?')[1:][0].split('&')
    genre = 'all'
    search_category = 'top'
    location = 0
    formatt = 'all'
    subgenre = 'all'
    week = 0
    for parameter in parameters:
        p = parameter.split('=')
        if p[0] == 'g':
            genre = p[1]
        if p[0] == 's':
            search_category = p[1]
        if p[0] == 'gn':
            location = int(p[1])
        if p[0] == 'f':
            formatt = p[1]
        if p[0] == 't':
            subgenre = p[1]
        if p[0] == 'w':
            week = int(p[1])
    
    print('scraping ' + category + ' ' + genre + ' (' + subgenre + ') albums from location \'' + str(location) + '\' in ' + formatt + ' format for week ' + str(week) + '...')
    
    entry = {'Release Genre': genre,
            'Release Sub-Genre': subgenre,
            'Search Format': formatt,
            'Search Week': WEEK_DICT[week],
            'Search Category': search_category}
    
    driver.get(URL)
    time.sleep(INITIAL_SEARCH_WAIT)
    
    # extract URLs from link tags
    release_URLS = []
    
    for i in range(num_pages):
        # this is just to ensure that the page is loaded
        
        html = driver.page_source

        # create bsoup object
        soup = bsoup(str(html), 'lxml')

        # find album link tags on this page
        link_tags = soup.find_all('a', {'class':'item-title'})
    
        for tag in link_tags:
            tag_url = tag['href'].split('?')[0]
            if tag_url not in release_URLS and tag_url not in all_URLS_collected:
                release_URLS.append(tag_url)
        
        try:
            driver.find_element_by_xpath("//a[contains(text(), 'next')]").click()
        except:
            break
            
    if output == True:
        pprint(release_URLS)

    for release_URL in release_URLS:
        scrape_release_page(release_URL,entry,output=output)

In [395]:
###################### USER INPUTS ######################################################

# list of genres to scrape
GENRES = ['all',
          'rock',
          #'metal',
          'alternative',
          'hip-hop-rap',
          'experimental',
          #'punk',
          'pop',
          'acoustic',
          #'funk',
          'country',
          'blues',
          #'ambient',
          #'soundtrack',
          #'world',
          'jazz',
          'r-b-soul',
          #'devotional',
          #'classical',
          'reggae'
          #'latin'
         ]

# scrape for subgenres within genres?
SCRAPE_SUBGENRES = False

# dictionary of subgenres to scrape, if scrape_subgenres = True
SUBGENRES = {
                'rock': ['indie','prog-rock','post-rock','rock-roll','psychedelic-rock'],
                'metal': [],
                'alternative': [],
                'hip-hop-rap': [],
                'experimental': [],
                'punk': [],
                'folk': [],
                'pop': [],
                'acoustic': [],
                'funk': [],
                'country': [],
                'blues': []
            }

# list of search categories to scrape ['top', 'new', 'rec']
#   top = best-selling
#   new = new arrivals
#   rec = artist-recommended
SEARCH_CATEGORIES = ['top','new']

# list of locations to scrape
# location = 0 returns search results for all locations
LOCATIONS = [0] 

# list of formats to scrape ['all','digital','vinyl','cd','cassette']
FORMATS = ['all']

# list of weeks to scrape
# valid weeks: [-1,0,678,677,676,675,674,673]
# today, this week, last week, 2, 3, 4, 5, 6 weeks ago
WEEKS = [673]

# include week parameter in search?
SCRAPE_WEEKS = True

# number of pages to scrape (10 produces 40-100 results)
NUM_PAGES = 13

# Initial time to wait (in seconds) for Bandcamp Discover page to load
INITIAL_SEARCH_WAIT = 5

# retrieve no more than this many fans for popularity index
MAX_FANS = 100000

########################################################################################

# max number of pages of fan results to scrape before hitting MAX_FANS
MAX_FAN_PAGES = int(MAX_FANS / 60)


In [ ]:
# main web scraper loop

for genre in GENRES:
    for category in SEARCH_CATEGORIES:
        for location in LOCATIONS:
            for formatt in FORMATS:
                for week in WEEKS:
                    if SCRAPE_WEEKS:
                        scrape_URL = 'https://bandcamp.com/?g=' + genre + '&s=' + category + '&p=0' + '&gn=' + str(location) + '&f=' + formatt + '&w=' + str(week)
                    else:
                        scrape_URL = 'https://bandcamp.com/?g=' + genre + '&s=' + category + '&p=0' + '&gn=' + str(location) + '&f=' + formatt
                        
                    scrape_search_page(scrape_URL,output=False)
                    
                    if SCRAPE_SUBGENRES and genre in SUBGENRES:
                        for subgenre in SUBGENRES[genre]:
                            subgenre_scrape_URL = scrape_URL + '&t=' + subgenre
                            scrape_search_page(subgenre_scrape_URL,output=False)

scraping top all (all) albums from location '0' in all format for week 673...


In [ ]:
# JOHN KING CAVE scrapes

#scrape_release_page('https://johnkingcave.bandcamp.com/album/devil-rides-beside',{'Release Genre':'folk'})
#scrape_release_page('https://johnkingcave.bandcamp.com/album/oh-my-love',{'Release Genre':'folk'})
#scrape_release_page('https://johnkingcave.bandcamp.com/album/emotion-tread-light',{'Release Genre':'folk'})
#scrape_release_page('https://johnkingcave.bandcamp.com/album/720-split',{'Release Genre':'folk'})
#scrape_release_page('https://johnkingcave.bandcamp.com/album/sing-a-song',{'Release Genre':'folk'})

# NOTE: Doesn't work for Bandcamp track pages
#scrape_release_page('https://johnkingcave.bandcamp.com/track/i-love-america',{'Release Genre':'folk'})
#scrape_release_page('https://johnkingcave.bandcamp.com/track/shes-coming-down-the-line-2',{'Release Genre':'folk'})
#scrape_release_page('https://johnkingcave.bandcamp.com/track/labyrinth-of-faith',{'Release Genre':'folk'})


In [ ]:
# Print Dataframe
scrape_df = pd.DataFrame(scrape_data)
scrape_df = scrape_df.drop_duplicates(subset = ["Release URL"])

scrape_df

In [ ]:
# save dataframe to CSV

timestamp = time.strftime('%b-%d-%Y_%H%M%S')
scrape_df.to_csv('scrape_data_' + timestamp + '.csv')


In [ ]:
# save collected URLS (to be loaded for next batch)
with open("all_URLS_collected.txt", "w") as fp:
    json.dump(all_URLS_collected, fp)